## 2.5 평가지표의 최적화

### 목차
 * 2.5.1 평가지표의 최적화 접근법
 * 2.5.2 임곗값 최적화

### 2.5.3 임곗값 최적화와 OOF 예측의 필요성

In [9]:
import numpy as np
import pandas as pd

In [10]:
# -----------------------------------
# out-of-fold에서의 임곗값(threshold)의 최적화
# -----------------------------------
from scipy.optimize import minimize
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

# 샘플 데이터 생성 준비
rand = np.random.RandomState(seed=71)
train_y_prob = np.linspace(0, 1.0, 10000)

# 실젯값과 예측값을 다음과 같은 train_y, train_pred_prob이었다고 가정
train_y = pd.Series(rand.uniform(0.0, 1.0, train_y_prob.size) < train_y_prob)
train_pred_prob = np.clip(train_y_prob * np.exp(
                       rand.standard_normal(train_y_prob.shape) * 0.3), 0.0, 1.0)


### 임곗값을 0.5로 하여 F1을 구하기

In [8]:
# 실젯값과 예측값을 다음과 같은 train_y, train_pred_prob이었다고 가정
train_y = pd.Series(rand.uniform(0.0, 1.0, train_y_prob.size) < train_y_prob)
train_pred_prob = np.clip(train_y_prob * np.exp(
                            rand.standard_normal(train_y_prob.shape) * 0.3), 0.0, 1.0)

# 임곗값(threshold)을 0.5로 하면, F1은 0.722
init_threshold = 0.5
init_score = f1_score(train_y, train_pred_prob >= init_threshold)
print(init_threshold, init_score)

0.5 0.7170388498011624


### 최적의 임계값을 기준으로 F1을 구하기

In [6]:
# 최적화의 목적함수를 설정
def f1_opt(x):
    return -f1_score(train_y, train_pred_prob >= x)

# scipy.optimize의 minimize 메소드에서 최적의 임곗값 구하기
# 구한 최적의 임곗값을 바탕으로 F1을 구하면 0.756이 됨
result = minimize(f1_opt, x0=np.array([0.5]), method="Nelder-Mead")
best_threshold = result['x'].item()
best_score = f1_score(train_y, train_pred_prob >= best_threshold)
print(best_threshold, best_score)

0.32324218749999983 0.7557317703844165
